In [1]:
# this walk-through was adapted from the following two tutorials, check them out for more info:
# https://machinelearningmastery.com/machine-learning-in-python-step-by-step/
# https://towardsdatascience.com/exploring-classifiers-with-python-scikit-learn-iris-dataset-2bcb490d2e1b


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [2]:
# games = pd.read_csv("data/root_games.csv")
games = pd.concat([pd.read_csv("data/root_games.csv"), pd.read_csv("data/tail_games1.csv"),
                   pd.read_csv("data/tail_games2.csv"), pd.read_csv("data/tail_games3.csv"),
                   pd.read_csv("data/tail_games4.csv")], axis=0, ignore_index=True)

/tmp/ipykernel_4080594/971427542.py:2: DtypeWarning: Columns (79,80,81,82,197,198,199,200,315,316,317,318,433,434,435,436,551,552,553,554,669,670,671,672,787,788,789,790,905,906,907,908,1023,1024,1025,1026,1141,1142,1143,1144) have mixed types. Specify dtype option on import or set low_memory=False.
  games = pd.concat([pd.read_csv("data/root_games.csv"), pd.read_csv("data/tail_games1.csv"),
/tmp/ipykernel_4080594/971427542.py:2: DtypeWarning: Columns (79,80,202,203,319,320,436,437,553,554,676,677,799,800,916,917,1039,1040,1156,1157) have mixed types. Specify dtype option on import or set low_memory=False.
  games = pd.concat([pd.read_csv("data/root_games.csv"), pd.read_csv("data/tail_games1.csv"),
/tmp/ipykernel_4080594/971427542.py:3: DtypeWarning: Columns (79,80,196,197,313,314,430,431,547,548,664,665,781,782,898,899,1015,1016,1132,1133) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv("data/tail_games2.csv"), pd.read_csv("data/tail_games3.csv"

In [3]:
games.head()

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,...,participant7.timeline.damageTakenDiffPerMinDeltas.30-end,participant8.timeline.csDiffPerMinDeltas.30-end,participant8.timeline.xpDiffPerMinDeltas.30-end,participant8.timeline.damageTakenDiffPerMinDeltas.30-end,participant9.timeline.csDiffPerMinDeltas.30-end,participant9.timeline.xpDiffPerMinDeltas.30-end,participant9.timeline.damageTakenDiffPerMinDeltas.30-end,participant10.timeline.csDiffPerMinDeltas.30-end,participant10.timeline.xpDiffPerMinDeltas.30-end,participant10.timeline.damageTakenDiffPerMinDeltas.30-end
0,3697668302,NA1,1607466298025,607,420,11,13,10.24.345.8390,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3684169604,NA1,1606615244343,1485,420,11,13,10.24.345.8390,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3683991059,NA1,1606603135732,1506,420,11,13,10.24.345.8390,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3674176170,NA1,1606011059740,1948,420,11,13,10.23.343.2581,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3670897053,NA1,1605828202468,1854,420,11,13,10.23.343.2581,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# games.value_counts()

In [5]:
wanted_data = ["participant1.stats.win",
    "participant1.championId", "participant1.spell1Id",
                "participant1.spell2Id", #"participant1.timeline.lane",
                "participant1.timeline.xpDiffPerMinDeltas.0-10", 
                "participant1.timeline.goldPerMinDeltas.0-10",
               "participant2.championId", "participant2.spell1Id",
                "participant2.spell2Id", #"participant2.timeline.lane",
                "participant2.timeline.xpDiffPerMinDeltas.0-10", 
                "participant2.timeline.goldPerMinDeltas.0-10",
               "participant3.championId", "participant3.spell1Id",
                "participant3.spell2Id", #"participant3.timeline.lane",
                "participant3.timeline.xpDiffPerMinDeltas.0-10", 
                "participant3.timeline.goldPerMinDeltas.0-10",
               "participant4.championId", "participant4.spell1Id",
                "participant4.spell2Id", #"participant4.timeline.lane",
                "participant4.timeline.xpDiffPerMinDeltas.0-10", 
                "participant4.timeline.goldPerMinDeltas.0-10",
               "participant5.championId", "participant5.spell1Id",
                "participant5.spell2Id", #"participant5.timeline.lane",
                "participant5.timeline.xpDiffPerMinDeltas.0-10", 
                "participant5.timeline.goldPerMinDeltas.0-10",
               "participant6.championId", "participant6.spell1Id",
                "participant6.spell2Id", #"participant6.timeline.lane",
                "participant6.timeline.xpDiffPerMinDeltas.0-10", 
                "participant6.timeline.goldPerMinDeltas.0-10",
               "participant7.championId", "participant7.spell1Id",
                "participant7.spell2Id", #"participant7.timeline.lane",
                "participant7.timeline.xpDiffPerMinDeltas.0-10", 
                "participant7.timeline.goldPerMinDeltas.0-10",
               "participant8.championId", "participant8.spell1Id",
                "participant8.spell2Id", #"participant8.timeline.lane",
                "participant8.timeline.xpDiffPerMinDeltas.0-10", 
                "participant8.timeline.goldPerMinDeltas.0-10",
               "participant9.championId", "participant9.spell1Id",
                "participant9.spell2Id", #"participant9.timeline.lane",
                "participant9.timeline.xpDiffPerMinDeltas.0-10", 
                "participant9.timeline.goldPerMinDeltas.0-10",
               "participant10.championId", "participant10.spell1Id",
                "participant10.spell2Id", #"participant10.timeline.lane",
                "participant10.timeline.xpDiffPerMinDeltas.0-10", 
                "participant10.timeline.goldPerMinDeltas.0-10"
                ]

In [6]:
df_game = games[wanted_data]

In [7]:
df_game = df_game.fillna(0)

In [8]:
df_game.head()

,participant1.stats.win,participant1.championId,participant1.spell1Id,participant1.spell2Id,participant1.timeline.xpDiffPerMinDeltas.0-10,participant1.timeline.goldPerMinDeltas.0-10,participant2.championId,participant2.spell1Id,participant2.spell2Id,participant2.timeline.xpDiffPerMinDeltas.0-10,...,participant9.championId,participant9.spell1Id,participant9.spell2Id,participant9.timeline.xpDiffPerMinDeltas.0-10,participant9.timeline.goldPerMinDeltas.0-10,participant10.championId,participant10.spell1Id,participant10.spell2Id,participant10.timeline.xpDiffPerMinDeltas.0-10,participant10.timeline.goldPerMinDeltas.0-10
0,True,79,4,11,115.06,304.9,147,4,12,115.06,...,145,7,4,-115.06,100.0,121,4,11,-115.06,222.9
1,True,202,4,7,-27.55,270.1,121,4,11,-150.20,...,28,11,4,150.20,425.7,12,14,4,27.55,180.5
2,True,412,14,4,-4.35,181.4,114,4,12,-109.80,...,20,11,4,6.70,264.1,67,7,4,4.35,303.3
3,False,246,4,11,-49.60,263.8,126,4,12,53.30,...,2,11,4,49.60,271.5,112,4,7,-32.20,284.7
4,False,163,4,7,1.10,333.2,62,4,12,125.90,...,24,4,12,-125.90,310.1,267,4,3,-1.10,232.9


In [9]:
# df_game.to_csv("data/processed_games.csv")

In [10]:
#divide into training and testing
train, test = train_test_split(df_game, test_size = .2, stratify = df_game['participant1.stats.win'], random_state = 0)


In [11]:
train.head()

,participant1.stats.win,participant1.championId,participant1.spell1Id,participant1.spell2Id,participant1.timeline.xpDiffPerMinDeltas.0-10,participant1.timeline.goldPerMinDeltas.0-10,participant2.championId,participant2.spell1Id,participant2.spell2Id,participant2.timeline.xpDiffPerMinDeltas.0-10,...,participant9.championId,participant9.spell1Id,participant9.spell2Id,participant9.timeline.xpDiffPerMinDeltas.0-10,participant9.timeline.goldPerMinDeltas.0-10,participant10.championId,participant10.spell1Id,participant10.spell2Id,participant10.timeline.xpDiffPerMinDeltas.0-10,participant10.timeline.goldPerMinDeltas.0-10
11469,False,84,12,4,68.40,275.2,106,11,4,-74.8,...,202,4,7,-12.20,279.1,80,14,4,-12.20,193.2
16132,True,55,14,4,-26.80,233.6,245,11,4,-26.8,...,114,12,4,26.80,339.4,117,3,4,26.80,151.4
3175,False,777,14,4,-53.00,247.3,154,4,11,37.9,...,523,4,7,-3.95,308.1,412,4,3,-3.95,184.6
15771,False,133,4,14,0.00,312.0,236,7,4,-45.0,...,92,4,12,-5.90,369.7,22,4,3,45.00,235.0
10482,True,202,7,4,25.35,338.8,76,4,11,0.0,...,106,12,4,3.90,318.3,64,11,4,0.00,283.7


In [12]:
#divide into x and y
x_train = train.drop(['participant1.stats.win'], axis=1)#fill in with the columns you want to predict
y_train = train['participant1.stats.win']

x_test = test.drop('participant1.stats.win', axis=1)# fill in with the same columns as x_train
y_test = test['participant1.stats.win']

In [13]:
def test_model(model):
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print("accuracy:", metrics.accuracy_score(y_pred,y_test))

In [60]:
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [14]:
test_model(
    LogisticRegression(solver='liblinear', multi_class='ovr')
)

accuracy: 0.7269351613824622


In [47]:
test_model(
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1, random_state=39)
)

accuracy: 0.681519565838332


In [37]:
test_model(
    KNeighborsClassifier(1000)
)

accuracy: 0.7003713224792917


In [53]:
test_model(
    GaussianNB()
)

accuracy: 0.7115109968580405


In [58]:
test_model(
    QuadraticDiscriminantAnalysis()
)

accuracy: 0.7052270779777207


/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [61]:
test_model(
    AdaBoostClassifier(random_state=39)
)

accuracy: 0.7135104255926878
